In [6]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np
import pandas as pd

import os
spark_home='/usr/local/spark-1.5.2-bin-2.7.1'
os.environ['SPARK_HOME']=spark_home
import sys
sys.path.append(os.path.join(spark_home, 'python'))
sys.path.append(os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark import SparkConf, SparkContext, StorageLevel
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
import os
import re
import pickle
from datetime import *
from csv import reader
from io import StringIO

In [27]:
sc.stop()
scfg=SparkConf()
scfg.set("spark.cores.max",32)
sc=SparkContext(master="spark://final-gateway:7077", appName="reddiculous", conf=scfg)
#sc=SparkContext(master="yarn-client", appName="reddiculous", )

In [28]:
sqlContext = SQLContext(sc)

# Get dataset record counts

In [29]:
df=sqlContext.read.parquet("hdfs://final-gateway/w251/reddit-comments.parquet")
sj_df=sqlContext.read.parquet("hdfs://final-gateway/w251/reddit-comments-selfjoin.parquet")
print "All: {1} cols, {0} rows".format(
    df.count(),
    len(df.take(1)[0].asDict()),
)
print "Self-joined: {1} cols, {0} rows".format(
    sj_df.count(),
    len(sj_df.take(1)[0].asDict()),
)

All: 33 cols, 1659361605 rows
Self-joined: 17 cols, 993838767 rows


In [30]:
sc.stop()

# Parse raw JSONL file from HDFS into a dataframe

In [ ]:
#raw_rdd=sc.textFile("hdfs://final-gateway/w251/reddit")
#parsed_df=sqlContext.jsonRDD(raw_rdd)
%time parsed_sub_df=sqlContext.read.json("hdfs://final-gateway/w251/reddit-submissions-100.jsonl.bz2")
parsed_sub_df.schema

In [ ]:
%time parsed_sub_df=sqlContext.read.json("hdfs://final-gateway/w251/reddit-submissions.json")

In [ ]:
%time parsed_sub_df.write.format("parquet").save("hdfs://final-gateway/w251/reddit-comments.parquet")

In [ ]:
%time parsed_df.count()

In [ ]:
%time parsed_df.take(1)

In [ ]:
parsed_df

# Save to Parquet file for faster reload (no text parsing)

In [ ]:
%time parsed_df.write.format("parquet").save("hdfs://final-gateway/w251/reddit.parquet")

In [ ]:
parsed_df

# Read saved reddit comment data

In [13]:
%time parq_df=sqlContext.read.parquet("hdfs://final-gateway/w251/reddit-comments.parquet")

CPU times: user 0 ns, sys: 6.68 ms, total: 6.68 ms
Wall time: 5.9 s


# Play with the dataframe

In [5]:
sr_counts=parq_df.groupBy(parq_df.subreddit).count().where('count>10000')
pop_sr=parq_df.join(sr_counts,'subreddit')
%time pop_sr.count()

CPU times: user 53.4 ms, sys: 7.77 ms, total: 61.2 ms
Wall time: 4min 46s


1619010650

In [14]:
%time parq_df.count()

CPU times: user 0 ns, sys: 2.73 ms, total: 2.73 ms
Wall time: 7.42 s


1659361605

In [ ]:
%time parq_df.rdd.map(lambda x: 1).reduce(lambda x,y: x+y)

In [ ]:
sqlContext.registerDataFrameAsTable(parq_df,"reddit")

In [ ]:
%time ret=sqlContext.sql("SELECT COUNT(*) FROM reddit")

In [ ]:
%time ret.take()

In [ ]:
%time parq_df.take(1)

# Shut down the spark connection

In [17]:
sc.stop()

# Try using an old GraphX wrapper WIP

In [16]:
import graphx

In [ ]:
vertex_rdd=graphx.VertexRDD(parq_df.rdd.map(lambda x: (hash(graphx.VertexId(x['id'])), x)))

In [ ]:
edge_rdd=parq_df.map(lambda x: (hash(x['id']), hash(x['parent_id'], 1.0)))

In [ ]:
graphx.Graph(vertex_rdd, edge_rdd)

In [ ]:
graphx.VertexRDD(parq_df.